# SciTeX Context Management

This comprehensive notebook demonstrates the SciTeX context module capabilities, covering context management, output suppression, and environment control utilities.

## Features Covered

### Output Control
* Output suppression utilities
* Quiet operation modes
* Context managers for clean execution

### Environment Management
* Temporary state changes
* Clean execution contexts
* Resource management

### Integration Examples
* Scientific computation workflows
* Data processing pipelines
* Automated analysis systems

In [ ]:
# Detect notebook name for output directory
import os
from pathlib import Path

# Get notebook name (for papermill compatibility)
notebook_name = "06_scitex_context"
if 'PAPERMILL_NOTEBOOK_NAME' in os.environ:
    notebook_name = Path(os.environ['PAPERMILL_NOTEBOOK_NAME']).stem


In [ ]:
import sys
sys.path.insert(0, '../src')
import scitex
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import warnings
import time
import os

# Set up example data directory
data_dir = Path("./context_examples")
data_dir.mkdir(exist_ok=True)


## Part 1: Basic Output Suppression

### 1.1 Suppress Output Context Manager

In [ ]:
# Demonstrate basic output suppression

# Normal output (visible)

# Suppressed output (hidden)

with scitex.context.suppress_output():
    
    # Even function calls that produce output
    for i in range(3):
        pass  # Loop body
def noisy_function():
    """A function that produces lots of output."""
    for i in range(5):
        pass  # Loop body
        time.sleep(0.01)
    return "Function result"

# Run function normally (noisy)
result1 = noisy_function()

# Run function with suppressed output (quiet)
with scitex.context.suppress_output():
    result2 = noisy_function()


### 1.2 Quiet Operation Mode

In [ ]:
# Demonstrate quiet operation mode

# Define functions with verbose output
def verbose_data_processing():
    """Simulate verbose data processing."""
    data = np.random.randn(1000, 50)
    
    normalized_data = (data - np.mean(data)) / np.std(data)
    
    correlations = np.corrcoef(normalized_data.T)
    
    eigenvalues, eigenvectors = np.linalg.eig(correlations)
    
    return {
    'data': normalized_data,
    'correlations': correlations,
    'eigenvalues': eigenvalues,
    'eigenvectors': eigenvectors
    }

def verbose_model_training():
    """Simulate verbose model training."""
    
    for epoch in range(10):
        loss = 1.0 / (epoch + 1) + 0.1 * np.random.random()
        accuracy = 1.0 - loss + 0.05 * np.random.random()
        time.sleep(0.01)  # Simulate training time
    
    return {'final_loss': loss, 'final_accuracy': accuracy}

# Test verbose operations
data_results = verbose_data_processing()

training_results = verbose_model_training()

# Test quiet operations using scitex.context.quiet

with scitex.context.quiet():
    quiet_data_results = verbose_data_processing()

with scitex.context.quiet():
    quiet_training_results = verbose_model_training()

# Verify results are identical

## Part 2: Advanced Context Management

### 2.1 Nested Context Managers

In [ ]:
# Demonstrate nested context managers

def multi_level_function():
    """Function with multiple levels of verbosity."""
    
    def level_2_function():
        for i in range(3):
            pass  # Loop body
        def level_3_function():
            for j in range(5):
                pass  # Loop body
            return "deep_result"
        
        result = level_3_function()
        return result
    
    result = level_2_function()
    return result

# Test normal execution
result1 = multi_level_function()

# Test single-level suppression
with scitex.context.suppress_output():
    result2 = multi_level_function()

# Test nested suppression contexts

def selective_suppression():
    
    with scitex.context.quiet():
        
        # Even more nested
        with scitex.context.suppress_output():
            for i in range(3):
                pass  # Loop body
    return "selective_result"

result3 = selective_suppression()

# Test context manager exception handling

def function_with_error():
    raise ValueError("Intentional error for testing")

# Test that context manager properly handles exceptions
try:
    with scitex.context.suppress_output():
        function_with_error()
except ValueError as e:
    pass  # Exception handled


### 2.2 Warning and Error Suppression

In [ ]:
# Demonstrate warning and error suppression

# Function that generates warnings
def function_with_warnings():
    """Function that generates various warnings."""
    
    # Generate numpy warnings
    
    # Division by zero warning
    with warnings.catch_warnings():
        warnings.simplefilter("always")
        result1 = np.array([1, 2, 3, 0]) / np.array([2, 0, 1, 0])  # Will generate warnings
    
    # Invalid value warning
    with warnings.catch_warnings():
        warnings.simplefilter("always")
        result2 = np.sqrt(np.array([-1, 4, -9, 16]))  # Will generate warnings
    
    # Overflow warning
    with warnings.catch_warnings():
        warnings.simplefilter("always")
        result3 = np.exp(np.array([700, 800, 900]))  # Will generate warnings
    
    
    return result1, result2, result3

# Test with warnings visible
with warnings.catch_warnings():
    warnings.simplefilter("always")  # Show all warnings
    results1 = function_with_warnings()

# Test with both output and warnings suppressed
with scitex.context.suppress_output():
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")  # Suppress warnings
        results2 = function_with_warnings()


# Verify results are still computed correctly

# Test stderr suppression

def function_with_stderr():
    """Function that writes to stderr."""
    sys.stderr.write("Writing to stderr\n")
    sys.stderr.write("Another stderr message\n")
    return "stderr_test_result"

# Normal execution (stderr visible)
result_normal = function_with_stderr()

# Suppressed execution
with scitex.context.suppress_output():
    result_suppressed = function_with_stderr()


## Part 3: Scientific Computing Applications

### 3.1 Clean Data Processing Pipelines

In [ ]:
# Clean data processing pipelines

class DataProcessor:
    """A data processor with verbose and quiet modes."""
    
    def __init__(self, verbose=True):
        self.verbose = verbose
        self.processing_log = []
    
    def log(self, message):
        """Log a message if verbose mode is enabled."""
        self.processing_log.append(message)
        if self.verbose:
            pass  # Condition handled
    
    def load_data(self, shape=(1000, 50)):
        """Load synthetic data."""
        self.log(f"Loading data with shape {shape}")
        data = np.random.randn(*shape)
        
        # Add some structure
        data[:, :10] += np.sin(np.linspace(0, 2*np.pi, shape[0]))[:, np.newaxis]
        data[:, 10:20] += np.cos(np.linspace(0, 4*np.pi, shape[0]))[:, np.newaxis]
        
        self.log(f"Data loaded successfully")
        self.log(f"Data statistics: mean={np.mean(data):.4f}, std={np.std(data):.4f}")
        
        return data
    
    def preprocess_data(self, data):
        """Preprocess the data."""
        self.log("Starting data preprocessing")
        
        # Step 1: Remove outliers
        self.log("Removing outliers (>3 std)")
        outlier_mask = np.abs(data) > 3 * np.std(data)
        data_cleaned = data.copy()
        data_cleaned[outlier_mask] = np.nan
        outliers_removed = np.sum(outlier_mask)
        self.log(f"Removed {outliers_removed} outliers")
        
        # Step 2: Interpolate missing values
        self.log("Interpolating missing values")
        for col in range(data_cleaned.shape[1]):
            mask = ~np.isnan(data_cleaned[:, col])
            if np.sum(mask) > 0:
                data_cleaned[~mask, col] = np.mean(data_cleaned[mask, col])
        
        # Step 3: Normalize
        self.log("Normalizing data (z-score)")
        data_normalized = (data_cleaned - np.mean(data_cleaned, axis=0)) / np.std(data_cleaned, axis=0)
        
        self.log("Preprocessing completed")
        self.log(f"Final data: mean={np.mean(data_normalized):.6f}, std={np.std(data_normalized):.6f}")
        
        return data_normalized
    
    def analyze_data(self, data):
        """Analyze the preprocessed data."""
        self.log("Starting data analysis")
        
        # Correlation analysis
        self.log("Computing correlation matrix")
        correlation_matrix = np.corrcoef(data.T)
        
        # Principal component analysis
        self.log("Performing PCA")
        eigenvalues, eigenvectors = np.linalg.eig(correlation_matrix)
        
        # Sort by eigenvalue magnitude
        idx = np.argsort(eigenvalues)[::-1]
        eigenvalues = eigenvalues[idx]
        eigenvectors = eigenvectors[:, idx]
        
        # Compute explained variance
        explained_variance = eigenvalues / np.sum(eigenvalues)
        cumulative_variance = np.cumsum(explained_variance)
        
        self.log(f"First 5 eigenvalues: {eigenvalues[:5]}")
        self.log(f"Variance explained by first 5 PCs: {explained_variance[:5]}")
        self.log(f"Cumulative variance (first 10 PCs): {cumulative_variance[9]:.4f}")
        
        # Cluster analysis
        self.log("Performing simple clustering")
        # Simple k-means-like clustering
        n_clusters = 3
        centroids = data[np.random.choice(data.shape[0], n_clusters, replace=False)]
        
        distances = np.sqrt(((data[:, np.newaxis, :] - centroids[np.newaxis, :, :]) ** 2).sum(axis=2))
        labels = np.argmin(distances, axis=1)
        
        cluster_sizes = [np.sum(labels == i) for i in range(n_clusters)]
        self.log(f"Cluster sizes: {cluster_sizes}")
        
        self.log("Analysis completed")
        
        return {
        'correlation_matrix': correlation_matrix,
        'eigenvalues': eigenvalues,
        'eigenvectors': eigenvectors,
        'explained_variance': explained_variance,
        'cumulative_variance': cumulative_variance,
        'cluster_labels': labels,
        'cluster_sizes': cluster_sizes
        }
    
    def run_pipeline(self, data_shape=(1000, 50)):
        """Run the complete data processing pipeline."""
        self.log("=" * 50)
        self.log("STARTING DATA PROCESSING PIPELINE")
        self.log("=" * 50)
        
        # Load data
        data = self.load_data(data_shape)
        
        # Preprocess
        processed_data = self.preprocess_data(data)
        
        # Analyze
        analysis_results = self.analyze_data(processed_data)
        
        self.log("=" * 50)
        self.log("PIPELINE COMPLETED SUCCESSFULLY")
        self.log("=" * 50)
        
        return {
        'raw_data': data,
        'processed_data': processed_data,
        'analysis': analysis_results,
        'log': self.processing_log
        }

# Test verbose pipeline
verbose_processor = DataProcessor(verbose=True)
verbose_results = verbose_processor.run_pipeline((500, 20))

# Test quiet pipeline using context manager

with scitex.context.quiet():
    quiet_processor = DataProcessor(verbose=True)  # Still verbose, but output suppressed
    quiet_results = quiet_processor.run_pipeline((500, 20))


# Compare results

# Show log comparison

### 3.2 Automated Analysis with Clean Output

In [ ]:
# Automated analysis with clean output

class AutomatedAnalyzer:
    """Automated analyzer that can run in quiet or verbose mode."""
    
    def __init__(self):
        self.analysis_history = []
    
    def analyze_dataset(self, dataset_name, data, quiet=False):
        """Analyze a dataset with optional quiet mode."""
        
        def verbose_analysis():
            
            
            # Basic statistics
            
            # Distribution analysis
            percentiles = [5, 25, 50, 75, 95]
            perc_values = np.percentile(data, percentiles)
            for p, v in zip(percentiles, perc_values):
        pass  # Processing p
            # Correlation analysis
            if data.ndim > 1 and data.shape[1] > 1:
                corr_matrix = np.corrcoef(data.T)
                
                # Find highest correlations
                mask = np.triu(np.ones_like(corr_matrix, dtype=bool), k=1)
                correlations = corr_matrix[mask]
                high_corr = correlations[np.abs(correlations) > 0.5]
                
                if len(high_corr) > 0:
                    # Condition met
            
            # Outlier detection
            z_scores = np.abs((data - np.mean(data)) / np.std(data))
            outliers = z_scores > 3
            n_outliers = np.sum(outliers)
            outlier_percentage = (n_outliers / data.size) * 100
            
            
            # Trend analysis
            if data.ndim == 1 or (data.ndim == 2 and data.shape[1] == 1):
                flat_data = data.flatten()
                x = np.arange(len(flat_data))
                slope, intercept = np.polyfit(x, flat_data, 1)
            
            
            # Return analysis results
            results = {
                'dataset_name': dataset_name,
                'shape': data.shape,
                'basic_stats': {
                'mean': np.mean(data),
                'std': np.std(data),
                'min': np.min(data),
                'max': np.max(data)
                },
                'percentiles': dict(zip(percentiles, perc_values)),
                'outliers': {
                'count': n_outliers,
                'percentage': outlier_percentage
                }
            }
            
            if data.ndim > 1 and data.shape[1] > 1:
                results['correlations'] = {
                'matrix_shape': corr_matrix.shape,
                'high_correlations': len(high_corr),
                'max_correlation': np.max(np.abs(high_corr)) if len(high_corr) > 0 else 0
                }
            
            if data.ndim == 1 or (data.ndim == 2 and data.shape[1] == 1):
                results['trend'] = {
                'slope': slope,
                'direction': 'Increasing' if slope > 0 else 'Decreasing' if slope < 0 else 'Flat'
                }
            
            return results
        
        # Run analysis with or without output suppression
        if quiet:
            with scitex.context.suppress_output():
                results = verbose_analysis()
        else:
            results = verbose_analysis()
        
        # Store in history
        self.analysis_history.append(results)
        
        return results
    
    def batch_analysis(self, datasets, quiet=True):
        """Perform batch analysis on multiple datasets."""
        
        results = []
        for name, data in datasets.items():
            if not quiet:
                # Condition met
            
            result = self.analyze_dataset(name, data, quiet=quiet)
            results.append(result)
            
            if not quiet:
                # Condition met
        
        return results
    
    def generate_summary(self):
        """Generate a summary of all analyses."""
        if not self.analysis_history:
            return
        
        
        # Summary statistics
        all_means = [r['basic_stats']['mean'] for r in self.analysis_history]
        all_stds = [r['basic_stats']['std'] for r in self.analysis_history]
        all_outlier_pcts = [r['outliers']['percentage'] for r in self.analysis_history]
        
        
        # Dataset with highest/lowest variation
        max_std_idx = np.argmax(all_stds)
        min_std_idx = np.argmin(all_stds)
        

# Create test datasets
test_datasets = {
    'random_normal': np.random.randn(1000, 10),
    'random_uniform': np.random.uniform(-1, 1, (800, 15)),
    'structured_sine': np.sin(np.linspace(0, 4*np.pi, 500)).reshape(-1, 1),
    'noisy_trend': np.linspace(0, 10, 1000) + 0.5 * np.random.randn(1000),
    'sparse_data': np.zeros((200, 20)),
}

# Add some structure to sparse data
test_datasets['sparse_data'][::10, ::5] = np.random.randn(20, 4)

# Create analyzer
analyzer = AutomatedAnalyzer()

# Test individual analysis (verbose)
individual_result = analyzer.analyze_dataset('test_normal', np.random.randn(100, 5), quiet=False)

# Test batch analysis (quiet)
batch_results = analyzer.batch_analysis(test_datasets, quiet=True)

# Generate summary
analyzer.generate_summary()

# Test mixed mode

with scitex.context.quiet():
    mixed_result = analyzer.analyze_dataset('mixed_mode', np.random.exponential(2, (300, 8)), quiet=False)


## Part 4: Performance and Resource Management

### 4.1 Performance Comparison with Context Managers

In [ ]:
# Performance comparison with context managers

import time

def performance_heavy_function(n_iterations=100):
    """A function that does heavy computation with lots of output."""
    results = []
    
    for i in range(n_iterations):
        pass  # Loop body
        data = np.random.randn(100, 100)
        
        # Matrix operations
        eigenvals = np.linalg.eigvals(data @ data.T)
        
        result = np.sum(eigenvals)
        results.append(result)
        
        if i % 10 == 9:
            pass  # Condition handled
    
    return results

# Test performance with output
start_time = time.time()
results_with_output = performance_heavy_function(10)
time_with_output = time.time() - start_time

# Test performance without output
start_time = time.time()
with scitex.context.suppress_output():
    results_without_output = performance_heavy_function(10)
time_without_output = time.time() - start_time

# Compare performance
if time_with_output > time_without_output:
    speedup = time_with_output / time_without_output
else:
    pass  # Else case

# Verify results are identical
results_match = np.allclose(results_with_output, results_without_output)

# Memory usage test

def memory_intensive_function():
    """Function that creates large objects and prints about them."""
    arrays = []
    
    for i in range(20):
        pass  # Loop body
        size = (i + 1) * 100
        arr = np.random.randn(size, size)
        arrays.append(arr)
        
        memory_usage = sum(a.nbytes for a in arrays) / (1024**2)  # MB
        
        if i % 5 == 4:
            pass  # Condition handled
    
    total_memory = sum(a.nbytes for a in arrays) / (1024**2)
    
    return arrays

# Test memory function with output
start_time = time.time()
arrays_with_output = memory_intensive_function()
time_memory_with = time.time() - start_time

# Clean up
del arrays_with_output

# Test memory function without output
start_time = time.time()
with scitex.context.suppress_output():
    arrays_without_output = memory_intensive_function()
time_memory_without = time.time() - start_time

# Compare memory test performance
memory_speedup = time_memory_with / time_memory_without if time_memory_without > 0 else 1

# Clean up
del arrays_without_output

### 4.2 Resource Management and Context Cleanup

In [ ]:
# Resource management and context cleanup

class ResourceManager:
    """Demonstrate resource management with context managers."""
    
    def __init__(self):
        self.resources = []
        self.resource_counter = 0
    
    def create_resource(self, name, size_mb=10):
        """Create a mock resource (large array)."""
        self.resource_counter += 1
        resource_id = f"{name}_{self.resource_counter}"
        
        # Create resource (large array)
        elements = int(size_mb * 1024 * 1024 / 8)  # 8 bytes per float64
        array_size = int(np.sqrt(elements))
        resource_data = np.random.randn(array_size, array_size)
        
        resource = {
        'id': resource_id,
        'name': name,
        'data': resource_data,
        'size_mb': resource_data.nbytes / (1024**2),
        'created_at': time.time()
        }
        
        self.resources.append(resource)
        
        return resource
    
    def cleanup_resources(self):
        """Clean up all resources."""
        total_memory = sum(r['size_mb'] for r in self.resources)
        count = len(self.resources)
        
        
        for resource in self.resources:
            del resource['data']  # Free the large array
        
        self.resources.clear()
    
    def get_memory_usage(self):
        """Get current memory usage."""
        total_mb = sum(r['size_mb'] for r in self.resources)
        return total_mb
    
    def resource_intensive_operation(self, n_resources=5):
        """Perform a resource-intensive operation."""
        
        for i in range(n_resources):
            resource = self.create_resource(f"data_array", size_mb=20)
            
            # Simulate processing
            
            # Some computation
            mean_value = np.mean(resource['data'])
            std_value = np.std(resource['data'])
            
            
            if i % 2 == 1:
                pass  # Condition handled
        
        final_memory = self.get_memory_usage()
        
        return final_memory

# Test resource management with output
manager1 = ResourceManager()
memory_used_1 = manager1.resource_intensive_operation(3)
manager1.cleanup_resources()

# Test resource management without output
manager2 = ResourceManager()

with scitex.context.suppress_output():
    memory_used_2 = manager2.resource_intensive_operation(3)


# Show current state

# Clean up with output
manager2.cleanup_resources()

# Test context manager exception handling with resources

class SafeResourceManager(ResourceManager):
    """Resource manager with automatic cleanup on exceptions."""
    
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type is not None:
            pass  # Condition handled
        else:
            pass  # Else case
        
        self.cleanup_resources()
        
        # Don't suppress the exception
        return False

# Test normal operation
with SafeResourceManager() as safe_manager:
    safe_manager.create_resource("test_resource", 5)
    safe_manager.create_resource("another_resource", 5)

# Test exception handling
try:
    with SafeResourceManager() as safe_manager:
        safe_manager.create_resource("test_resource", 5)
        
        # Cause an intentional exception
        raise ValueError("Intentional error for testing")
        
except ValueError as e:
    pass  # Exception handled

# Test nested context managers

with SafeResourceManager() as outer_manager:
    outer_manager.create_resource("outer_resource", 10)
    
    with scitex.context.suppress_output():
        outer_manager.create_resource("quiet_resource_1", 10)
        outer_manager.create_resource("quiet_resource_2", 10)
        
        # This output will be suppressed


## Summary and Best Practices

This tutorial demonstrated the comprehensive context management capabilities of the SciTeX context module:

### Key Features Covered:
1. **Output Suppression**: `suppress_output()` for clean execution
2. **Quiet Operations**: `quiet()` for silent processing
3. **Context Management**: Proper resource handling and cleanup
4. **Exception Safety**: Robust error handling with context managers
5. **Performance Optimization**: Reduced overhead from suppressed output
6. **Nested Contexts**: Complex workflow management
7. **Resource Management**: Memory and resource cleanup
8. **Scientific Applications**: Clean data processing pipelines

### Best Practices:
- Use **output suppression** for batch processing and automated workflows
- Apply **quiet operations** when running repetitive analyses
- Implement **proper exception handling** in context managers
- Use **nested contexts** for complex processing pipelines
- Apply **resource management** for memory-intensive operations
- Use **context managers** for temporary state changes
- Implement **clean interfaces** that can operate in silent mode
- Consider **performance benefits** of suppressing verbose output

### Recommended Workflows:
1. **Batch Processing**: Use quiet mode for multiple dataset analysis
2. **Automated Pipelines**: Suppress output during production runs
3. **Interactive Development**: Use normal mode for debugging, quiet for final runs
4. **Resource Management**: Implement context managers for cleanup
5. **Performance Optimization**: Profile with and without output suppression

### Context Manager Patterns:
```python
# Basic suppression
with scitex.context.suppress_output():
    noisy_function()

# Quiet operations
with scitex.context.quiet():
    batch_process_data()

# Resource management
with ResourceManager() as manager:
    manager.process_data()
    # Automatic cleanup on exit
```

In [ ]:
cleanup = False  # Set to True to remove example files
# Cleanup
import shutil

# cleanup = "n"  # input("Clean up example files? (y/n): ").lower().startswith('y')
if cleanup:
    shutil.rmtree(data_dir)
else:
    if data_dir.exists():
        files = list(data_dir.rglob('*'))
        
        if files:
            total_size = sum(f.stat().st_size for f in files if f.is_file())
